In [0]:
import re
import pickle
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.preprocessing import image
from keras.utils import np_utils

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
project_dir = "gdrive/My Drive/metis_project_5"

In [0]:
# Get easy labels

# with open('labels/classifier/easy_labels.csv') as f:
#     raw_easy_labels = f.read().split('\n')[1:-1]

# easy_label_regex = re.compile('^([^\d]*)([^.]*)\.png,(.*)$')

# easy_label_to_int = {'a': 0, 'g': 1, 'db': 2, 'ub': 3, 's': 4}

# easy_paths, easy_y = [], []
# for label in raw_easy_labels:
#     (category, frame, label) = label_regex.match(label).groups()
#     int_label = label_to_int.get(label)
#     if int_label and category == 'new_fox_':
#         path = f"data/larger_cropped/{frame}.png"
#         easy_paths.append(path)
#         easy_y.append(int_label)

# easy_num_categories = 5

In [56]:
# Get new easy_larger_cropped labels, wiz_sfat

with open(f"{project_dir}/labels/classifier/easy_wiz_sfat_larger_cropped_labels.csv") as f:
    raw_ws_labels = f.read().split('\n')[1:-1]

ws_label_regex = re.compile('^([^.]*)\.png,(.*)$')

ws_label_to_int = {
    'g': 0,
    'a': 1,
    's': 0,
    'aub': 1,
    'gdb': 0,
    'adb': 1,
    # 'l': 1,
}

ws_paths, ws_labels, ws_y = [], [], []
for label in raw_ws_labels:
    frame, label = ws_label_regex.match(label).groups()
    ws_labels.append(label)
    if label in ws_label_to_int.keys():
        int_label = ws_label_to_int[label]
        path = f"{project_dir}/da0ta/wiz_sfat/larger_cropped/{frame}.png"
        ws_paths.append(path)
        ws_y.append(int_label)

ws_num_categories = 2
print(Counter(ws_labels))

Counter({'g': 200, 'a': 128, 's': 7, 'gdb': 5, 'aub': 5, 'adb': 2, 'l': 1})


In [75]:
# Get easy labels, tbh7_purp_fox_vgbc

with open(f"{project_dir}/labels/classifier/easy_tbh7_purp_fox_vgbc_labels.csv") as f:
    raw_tbh_labels = f.read().split('\n')[1:-1]

tbh_label_regex = re.compile('^([^.]*)\.png,(.*)$')

tbh_label_to_int = {
    'g': 0,
    'a': 1,
    's': 0,
    'aub': 1,
    'gdb': 0,
    'adb': 1,
    # 'l': 1,
}

tbh_paths, tbh_labels, tbh_y = [], [], []
for label in raw_tbh_labels:
    frame, label = tbh_label_regex.match(label).groups()
    tbh_labels.append(label)
    if label in tbh_label_to_int.keys():
        int_label = tbh_label_to_int[label]
        path = f"{project_dir}/data/tbh7_purp_fox_vgbc/larger_cropped/{frame}.png"
        tbh_paths.append(path)
        tbh_y.append(int_label)

tbh_num_categories = 2
print(Counter(tbh_labels))

Counter({'g': 187, 'a': 173, 'aub': 18, 'l': 10, 's': 6, 'adb': 3, 'gdb': 3})


In [0]:
# Get hard labels

# with open('labels/classifier/labels.csv') as f:
#     raw_hard_labels = f.read().split('\n')[1:-1]

# hard_label_regex = re.compile('^([^\d]*)([^.]*)\.png,.,(.*)$')

# hard_label_to_int = {
#     'a': 0,
#     'adb': 1,
#     'anb': 2,
#     'aub': 3,
#     'ba': 4,
#     'd': 5,
#     'da': 6,
#     'dt': 7,
#     'g': 8,
#     'gdb': 9,
#     'h': 10,
#     'ha': 11,
#     'hg': 12,
#     'j': 13,
#     'l': 14,
#     'na': 15,
#     'p': 16,
#     's': 17,
#     'sd': 18,
#     't': 19,
#     'ua': 20,
#     'ut': 21,
# }

# hard_paths, hard_y = [], []
# for label in raw_hard_labels:
#     (category, frame, label) = hard_label_regex.match(label).groups()
#     int_label = label_to_int.get(label)
#     if int_label and category == 'new_fox_':
#         labels.add(label)
#         path = f"data/larger_cropped/{frame}.png"
#         paths.append(path)
#         hard_y.append(label)

# hard_num_categories = 22

In [0]:
# Choose labels

paths = ws_paths
y = ws_y
num_categories = ws_num_categories

In [36]:
# image augmentation
train_datagen = image.ImageDataGenerator(
    # rescale=1./255,
    featurewise_std_normalization=True,
    vertical_flip=True,
    validation_split=0.25)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
# only applying scaling to the validation set
# val_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
# training and validation gerators
train_gen = train_datagen.flow(X_train, y_train_cat)
val_gen = val_datagen.flow(X_val, y_val_cat)
epochs=100
# callbacks
es = EarlyStopping(monitor='val_loss', mode='min', patience=30, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1, factor=0.2,
                              patience=10, cooldown=5, min_lr=0.0002)
NN.summary()
NN.fit_generator(train_gen, steps_per_epoch=100, epochs=epochs, callbacks=[es, reduce_lr], 
                 verbose=1, validation_data=val_gen, validation_steps=20)

In [0]:
img_hw = (350, 350)
img_dimensions = (350, 350, 3)

def prepare_image(img_path):
    img = image.load_img(img_path, target_size=img_hw)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    # x = mobilenet_v2.preprocess_input(x)
    return x

In [0]:
# Prepare test/train data, wiz_sfat

ws_X = np.array([prepare_image(path)[0] for path in ws_paths])

ws_X_tr, ws_X_te, ws_y_tr, ws_y_te = train_test_split(ws_X, ws_y, test_size=0.2, random_state=42)

ws_y_tr_cat = np_utils.to_categorical(ws_y_tr)

In [76]:
# Prepare test/train data, tbh7

tbh_X = np.array([prepare_image(path)[0] for path in tbh_paths])

tbh_X_tr, tbh_X_te, tbh_y_tr, tbh_y_te = train_test_split(tbh_X, tbh_y, test_size=0.2, random_state=42)

tbh_y_tr_cat = np_utils.to_categorical(tbh_y_tr)

FileNotFoundError: ignored

In [63]:
pickles_dir = project_dir + "/pickles"

!ls gdrive/My\ Drive/metis_project_5/pickles

model_1.pickle	X_tr.pickle  y_tr_cat.pickle
X_te.pickle	y_te.pickle  y_tr.pickle


In [0]:
# with open(f'{pickles_dir}/ws_X_tr.pickle', 'wb') as f:
#     pickle.dump(ws_X_tr, f)

# with open(f'{pickles_dir}/ws_X_te.pickle', 'wb') as f:
#     pickle.dump(ws_X_te, f)

# with open(f'{pickles_dir}/ws_y_tr.pickle', 'wb') as f:
#     pickle.dump(ws_y_tr, f)

# with open(f'{pickles_dir}/ws_y_tr_cat.pickle', 'wb') as f:
#     pickle.dump(ws_y_tr_cat, f)

# with open(f'{pickles_dir}/ws_y_te.pickle', 'wb') as f:
#     pickle.dump(ws_y_te, f)

In [0]:
with open(f'{pickles_dir}/ws_X_tr.pickle', 'rb') as f:
    ws_X_tr = pickle.load(f)

with open(f'{pickles_dir}/ws_X_te.pickle', 'rb') as f:
    ws_X_te = pickle.load(f)

with open(f'{pickles_dir}/ws_y_tr.pickle', 'rb') as f:
    ws_y_tr = pickle.load(f)

with open(f'{pickles_dir}/ws_y_tr_cat.pickle', 'rb') as f:
    ws_y_tr_cat = pickle.load(f)

with open(f'{pickles_dir}/ws_y_te.pickle', 'rb') as f:
    ws_y_te = pickle.load(f)

In [0]:
# Get pretrained model

pretrained = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=img_dimensions,
)

In [0]:
# Build model

for layer in pretrained.layers[:17]:
    layer.trainable = False

model = Sequential()
model.add(pretrained)
model.add(Flatten())

if num_categories > 2:
    model.add(Dense(num_categories))
    loss='categorical_crossentropy',
else:
    model.add(Dense(1, activation='sigmoid'))
    loss='binary_crossentropy',

model.compile(
    loss=loss,
    optimizer='adam',
    metrics=['accuracy'],
)

In [88]:
# Train model

for _ in range(10):
  if num_categories > 2:
      model.fit(X_tr, y_tr_cat)
  else:
      model.fit(X_tr, y_tr)
      y_pr = model.predict_classes(X_te)[:,0]
      print(f"Test accuracy: {len(y_pr[y_pr == y_te]) / len(y_pr)}")

Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss: 0.8413 - accuracy: 0.9170
Test accuracy: 0.8
Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss: 0.1351 - accuracy: 0.9747
Test accuracy: 0.8571428571428571
Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss: 0.0772 - accuracy: 0.9928
Test accuracy: 0.8285714285714286
Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss: 0.0118 - accuracy: 0.9964
Test accuracy: 0.8285714285714286
Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss: 1.1469e-05 - accuracy: 1.0000
Test accuracy: 0.8428571428571429
Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss: 5.5487e-05 - accuracy: 1.0000
Test accuracy: 0.8571428571428571
Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss: 6.7488e-06 - accuracy: 1.0000
Test accuracy: 0.8571428571428571
Epoch 1/1
277/277 [==============================] - 2s 7ms/step - loss:

In [0]:
# with open('pickles/model_1.pickle', 'wb') as f:
    # pickle.dump(model, f)

In [0]:
with open('pickles/model_1.pickle', 'rb') as f:
    model = pickle.load(f)

In [87]:
print(f"Train zeros: {len(list(filter(lambda x: x == 0, y_tr)))}")
print(f"Train ones: {len(list(filter(lambda x: x == 1, y_tr)))}")
print(f"Test zeros: {len(list(filter(lambda x: x == 0, y_te)))}")
print(f"Test ones: {len(list(filter(lambda x: x == 1, y_te)))}")

Train zeros: 168
Train ones: 109
Test zeros: 44
Test ones: 26


In [79]:
for x in zip(y_pr, y_te):
  print(x)

(0, 0)
(0, 0)
(0, 0)
(0, 1)
(1, 0)
(0, 0)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(1, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(0, 0)
(0, 1)
(1, 1)
(1, 1)
(0, 0)
(1, 1)
(0, 0)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(0, 0)
(0, 1)
(0, 0)
(0, 0)
(1, 0)
(1, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(0, 0)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(0, 0)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 0)
(0, 0)
(1, 1)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 1)
(0, 0)
(0, 0)


In [78]:
# Predict and score

y_pr = model.predict_classes(X_te)[:,0]

print(f"Test accuracy: {len(y_pr[y_pr == y_te]) / len(y_pr)}")

assert(len(y_pr) == len(y_te))

y_tr_pr = model.predict_classes(X_tr)[:,0]

print(f"Train accuracy: {len(y_tr_pr[y_tr_pr == y_tr]) / len(y_tr_pr)}")

assert(len(y_tr_pr) == len(y_tr))

Test accuracy: 0.8285714285714286
Train accuracy: 0.9566787003610109


In [75]:
print(y_pr)
print(y_te)
print(y_tr_pr)
print(y_tr)

[0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 1 0
 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0]
[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0]
[[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 

In [0]:
model.save_weights?